#  Analyse de l'Évolution du Trafic SNCF par Région

## Objectifs
1. Analyser les tendances temporelles du trafic voyageurs
2. Évaluer l'impact COVID-19 et la reprise par région
3. Identifier les régions les plus dynamiques
4. Étudier les variations saisonnières

## Table des matières
1. [Configuration initiale](#1)
2. [Préparation des données](#2)
3. [Évolution globale du trafic](#3)
4. [Analyse régionale](#4)
5. [Impact COVID-19](#5)
6. [Analyse de la reprise](#6)
7. [Conclusions](#7)

<a id='1'></a>
## 1. Configuration initiale

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration des visualisations
%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
sns.set_palette('husl')

# Chemins des fichiers
DATA_PATH = Path('../data/processed/gares_avec_clusters.csv')

<a id='2'></a>
## 2. Préparation des données

In [ ]:
# Chargement des données
df = pd.read_csv(DATA_PATH)

# Création des colonnes d'évolution
df['evolution_2015_2019'] = (
    (df['total_voyageurs_2019'] - df['total_voyageurs_2015']) 
    / df['total_voyageurs_2015'] * 100
)

df['evolution_2019_2020'] = (
    (df['total_voyageurs_2020'] - df['total_voyageurs_2019']) 
    / df['total_voyageurs_2019'] * 100
)

df['evolution_2020_2023'] = (
    (df['total_voyageurs_2023'] - df['total_voyageurs_2020']) 
    / df['total_voyageurs_2020'] * 100
)

print("Aperçu des données d'évolution :")
print(df[['nom_de_la_gare', 'evolution_2015_2019', 'evolution_2019_2020', 'evolution_2020_2023']].head())

<a id='3'></a>
## 3. Évolution globale du trafic

In [ ]:
# Calcul du trafic total par année
yearly_total = pd.DataFrame({
    'année': range(2015, 2024),
    'total_voyageurs': [df[f'total_voyageurs_{year}'].sum() for year in range(2015, 2024)]
})

# Visualisation de l'évolution
plt.figure(figsize=(12, 6))
plt.plot(yearly_total['année'], yearly_total['total_voyageurs'] / 1e6, marker='o')
plt.title('Évolution du trafic total des gares SNCF (2015-2023)')
plt.xlabel('Année')
plt.ylabel('Millions de voyageurs')
plt.grid(True)

# Ajout des annotations pour les événements importants
plt.annotate('COVID-19', xy=(2020, yearly_total.loc[yearly_total['année']==2020, 'total_voyageurs'].values[0]/1e6),
             xytext=(2020.2, yearly_total['total_voyageurs'].max()/1e6*0.8),
             arrowprops=dict(facecolor='red', shrink=0.05))

plt.show()

<a id='4'></a>
## 4. Analyse régionale

In [ ]:
# Agrégation par région
regional_evolution = df.groupby('region').agg({
    'evolution_2015_2019': 'mean',
    'evolution_2019_2020': 'mean',
    'evolution_2020_2023': 'mean',
    'total_voyageurs_2023': 'sum'
}).round(2)

# Visualisation des évolutions régionales
plt.figure(figsize=(15, 8))
sns.heatmap(regional_evolution[['evolution_2015_2019', 'evolution_2019_2020', 'evolution_2020_2023']],
            annot=True, cmap='RdYlBu', center=0)
plt.title('Évolution du trafic par région (%)')
plt.tight_layout()
plt.show()

print("\nClassement des régions par trafic 2023 :")
print(regional_evolution.sort_values('total_voyageurs_2023', ascending=False))

<a id='5'></a>
## 5. Impact COVID-19

In [ ]:
# Distribution de l'impact COVID par région
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='region', y='evolution_2019_2020')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution de l\'impact COVID-19 par région')
plt.ylabel('Évolution 2019-2020 (%)')
plt.tight_layout()
plt.show()

# Top 10 des gares les plus impactées
print("\nTop 10 des gares les plus impactées par le COVID-19 :")
print(df.nsmallest(10, 'evolution_2019_2020')[['nom_de_la_gare', 'region', 'evolution_2019_2020']])

<a id='6'></a>
## 6. Analyse de la reprise

In [ ]:
# Calcul du taux de reprise (2023 vs 2019)
df['taux_reprise'] = (df['total_voyageurs_2023'] / df['total_voyageurs_2019'] - 1) * 100

# Visualisation de la reprise par région
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='region', y='taux_reprise', ci=95)
plt.xticks(rotation=45, ha='right')
plt.title('Taux de reprise par région (2023 vs 2019)')
plt.ylabel('Taux de reprise (%)')
plt.axhline(y=0, color='r', linestyle='--')
plt.tight_layout()
plt.show()

# Top 10 des meilleures reprises
print("\nTop 10 des gares avec la meilleure reprise :")
print(df.nlargest(10, 'taux_reprise')[['nom_de_la_gare', 'region', 'taux_reprise']])

In [16]:
# Calculer les moyennes des variations
baisse_moyenne_2020 = df_impact['variation_2020_%'].mean()
variation_moyenne_2023 = df_impact['variation_2023_%'].mean()

print(f"Impact moyen de la crise COVID (2020) : {baisse_moyenne_2020:.1f}%")
print(f"État de la reprise en 2023 : {variation_moyenne_2023:.1f}%")
print("\nInterprétation :")
print(f"- En 2020, le trafic a connu une baisse moyenne de {abs(baisse_moyenne_2020):.1f}% par rapport à la période pré-COVID.")

if variation_moyenne_2023 > 0:
    print(f"- En 2023, le trafic montre une reprise avec une hausse moyenne de {variation_moyenne_2023:.1f}% par rapport à la période pré-COVID.")
else:
    print(f"- En 2023, le trafic reste inférieur de {abs(variation_moyenne_2023):.1f}% à la période pré-COVID.")

# Compter le nombre de zones ayant retrouvé leur niveau pré-COVID
zones_retablies = (df_impact['variation_2023_%'] >= 0).sum()
print(f"\nSur les {len(df_impact)} zones analysées, {zones_retablies} ont retrouvé ou dépassé leur niveau de trafic pré-COVID en 2023.")

Impact moyen de la crise COVID (2020) : -48.0%
État de la reprise en 2023 : -6.1%

Interprétation :
- En 2020, le trafic a connu une baisse moyenne de 48.0% par rapport à la période pré-COVID.
- En 2023, le trafic reste inférieur de 6.1% à la période pré-COVID.

Sur les 10 zones analysées, 2 ont retrouvé ou dépassé leur niveau de trafic pré-COVID en 2023.
